In [1]:
import numpy as np
import pandas as pd

In [2]:
DATASETS = [["acm", 10], ["20ng", 10], ["webkb", 10]]

CLFS_SET = [
    ["bert", "temperature_scaling"],
    ["xlnet", "temperature_scaling"],
    ["ktmk", "isotonic"],
    ["ktr", "isotonic"],
    ["lstmk", "isotonic"],
    ["lstr", "isotonic"],
    ["ltr", "isotonic"]
]
CLFS_SET.sort(key=lambda x: x[0])

FEATURES_SET = [
    "probas-based"
]

FEATURES_SET.sort()
STACKING_DIR = "/home/welton/data/stacking/stacking_output"

idx_to_clf = {idx: clf for idx, (clf, _) in enumerate(CLFS_SET)}
clf_sufix = '/'.join(sorted([ f"{c[0]}_{c[1]}" for c in CLFS_SET ]))
mf_sufix = '_'.join(sorted(FEATURES_SET))
clf_sufix, mf_sufix

('bert_temperature_scaling/ktmk_isotonic/ktr_isotonic/lstmk_isotonic/lstr_isotonic/ltr_isotonic/xlnet_temperature_scaling',
 'probas-based')

In [3]:
output = []
for dataset, n_folds in DATASETS:
    rows = []
    for fold in np.arange(n_folds):
        base_dir = f"{STACKING_DIR}/{dataset}/{n_folds}_folds/logistic_regression/hard_docs_upper_bound/{clf_sufix}/{mf_sufix}/fold_{fold}"
        loader = np.load(f"{base_dir}/output.npz")

        loader["test_hard_docs"]
        y = loader["y_test"]
        hard_docs = loader["test_hard_docs"]
        spec_preds = loader["spec_preds"]
        preds = loader["normal_preds"]
        hards = hard_docs == 1
        softs = hard_docs == 0
        rows.append([np.sum(hards),
            np.sum(spec_preds[hards] == y[hards]),
            np.sum(preds[hards] == y[hards]),
            np.sum(softs),
            np.sum(spec_preds[softs] == y[softs]),
            np.sum(preds[softs] == y[softs])])

    output.append([dataset] + np.sum(rows, axis=0).tolist())
    

In [4]:
pd.DataFrame(output)

,0,1,2,3,4,5,6
0,acm,5020,1630,1107,19877,7831,19571
1,20ng,1904,669,629,16942,10251,16777
2,webkb,1358,781,565,6841,2591,6664


In [5]:
pd.DataFrame(rows, columns=["N Hards", "hard_spec", "hard_trad", "N Softs", "soft_spec", "soft_trad"])

,N Hards,hard_spec,hard_trad,N Softs,soft_spec,soft_trad
0,131,74,62,692,242,666
1,143,86,67,679,266,661
2,143,80,52,678,256,667
3,138,83,59,683,282,656
4,128,66,43,693,281,679
5,162,67,58,658,189,638
6,125,89,62,694,270,685
7,141,89,59,677,274,656
8,111,66,51,706,259,693
9,136,81,52,681,272,663


In [10]:
x = np.random.uniform(0, 1, size=(5,2))
x

array([[0.81399035, 0.06701792],
       [0.44520879, 0.72833499],
       [0.78782988, 0.08904669],
       [0.20091712, 0.28511868],
       [0.21630081, 0.32462673]])

In [11]:
x[:, 1]

array([0.06701792, 0.72833499, 0.08904669, 0.28511868, 0.32462673])